# Analysis

This notebook explores the data in the traffic dataset provided by ETHZ.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as cx

In [2]:
# Load the data
detectors = pd.read_csv('dataset/detectors.csv')
links     = pd.read_csv('dataset/links.csv')
traffic   = pd.read_csv('dataset/smalltraffic.csv', low_memory=False)

In [3]:
detectors['det_id']  = detectors['detid']
detectors['link_id'] = detectors['linkid']
detectors['length']  = detectors['length']
detectors['type']    = detectors['fclass']
detectors['city']    = detectors['citycode']

detectors = detectors.drop(columns=['detid', 'linkid', 'length', 'fclass', 'citycode'])

In [4]:
links['link_id']  = links['linkid']
links['city'] = links['citycode']

links = links.drop(columns=['linkid', 'citycode', 'piece', 'group'])

In [5]:
traffic['interval'] = traffic['interval'].apply(lambda x: f'{x // 3600}:{x % 3600 // 60}:{x % 60}')
traffic['datetime'] = pd.to_datetime(traffic['day'] + ' ' + traffic['interval'], format='%Y-%m-%d %H:%M:%S')
traffic['det_id'] = traffic['detid']

traffic = traffic.drop(columns=['day', 'interval', 'detid', 'error'])

In [6]:
detectors = detectors[['det_id', 'link_id', 'city', 'road', 'type', 'lanes', 'limit', 'lat', 'long', 'pos']]
detectors.head()

,det_id,link_id,city,road,type,lanes,limit,lat,long,pos
0,U1-52G,72.0,augsburg,Gögginger Straße,secondary,1.0,50,48.359957,10.889553,0.005512
1,U1-51G,73.0,augsburg,Gögginger Straße,secondary,1.0,50,48.359945,10.889601,0.004013
2,U1-52L,70.0,augsburg,Gögginger Straße,secondary,1.0,50,48.359876,10.889356,0.022228
3,U1-51L,71.0,augsburg,Gögginger Straße,secondary,1.0,50,48.359862,10.889396,0.021889
4,U1-62,68.0,augsburg,Rosenaustraße,secondary,1.0,50,48.360578,10.889361,0.024465


In [7]:
links = links[['link_id', 'order', 'city', 'lat', 'long']]
links.head()

,link_id,order,city,lat,long
0,0,1,augsburg,48.361079,10.891016
1,0,2,augsburg,48.360993,10.890894
2,0,3,augsburg,48.360853,10.890642
3,0,4,augsburg,48.360719,10.890404
4,0,5,augsburg,48.360497,10.889994


In [8]:
traffic = traffic[['det_id', 'datetime', 'city', 'flow', 'occ', 'speed']]
traffic.head()

,det_id,datetime,city,flow,occ,speed
0,06.X-2li,2017-05-06 00:00:00,augsburg,12,0.0,NaN
1,06.X-2li,2017-05-06 00:05:00,augsburg,12,0.0,NaN
2,06.X-2li,2017-05-06 00:10:00,augsburg,12,0.0,NaN
3,06.X-2li,2017-05-06 00:15:00,augsburg,16,0.0,NaN
4,06.X-2li,2017-05-06 00:20:00,augsburg,16,0.0,NaN


In [9]:
cities = detectors['city'].unique()

# Get the available columns out of ['flow', 'occ', 'speed'] for each city
available_columns = {}
for city in cities:
    # Load {city}.csv
    city_data = pd.read_csv(f'dataset/cities/{city}.csv')

    # Check which of the columns ['flow', 'occ', 'speed'] are not empty
    available_columns[city] = city_data[['flow', 'occ', 'speed']].notnull().any()

    print(f'{city}: {'flow' if available_columns[city]["flow"] else ''} {'occ' if available_columns[city]["occ"] else ''} {'speed' if available_columns[city]["speed"] else ''}')

augsburg: flow occ 
basel: flow occ 
bern: flow occ 
birmingham: flow  speed
bolton: flow occ speed
bordeaux: flow occ 
bremen: flow occ 
cagliari: flow occ 
constance: flow occ speed
darmstadt: flow occ 
essen: flow occ speed
frankfurt: flow occ 
graz: flow occ 
groningen: flow occ speed
hamburg: flow occ 
innsbruck: flow  speed
kassel: flow occ 
london: flow occ 
losangeles: flow occ 
luzern: flow occ 
madrid: flow occ 
melbourne: flow  
manchester: flow occ speed
marseille: flow occ 
munich: flow occ 
paris: flow occ 
rotterdam: flow occ speed
santander: flow occ 
speyer: flow occ 
strasbourg: flow occ 


/var/folders/9j/ddzkzm7x2bv5txk3k7qblr_80000gn/T/ipykernel_40335/3587162658.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  city_data = pd.read_csv(f'dataset/cities/{city}.csv')


stuttgart: flow occ 
taipeh: flow occ 
tokyo:   
torino: flow occ speed
toronto: flow occ 
toulouse: flow occ 
utrecht: flow  
vilnius: flow occ 
wolfsburg: flow occ 
zurich: flow occ 


In [11]:
# Pick out the cities with flow and occupancy data
cities_with_flow_occ = [city for city in cities if available_columns[city]['flow'] and available_columns[city]['occ']]
cities_with_flow_occ.remove('losangeles')
cities_with_flow_occ.remove('toronto')
cities_with_flow_occ.remove('taipeh')
print(cities_with_flow_occ)

['augsburg', 'basel', 'bern', 'bolton', 'bordeaux', 'bremen', 'cagliari', 'constance', 'darmstadt', 'essen', 'frankfurt', 'graz', 'groningen', 'hamburg', 'kassel', 'london', 'luzern', 'madrid', 'manchester', 'marseille', 'munich', 'paris', 'rotterdam', 'santander', 'speyer', 'strasbourg', 'stuttgart', 'torino', 'toulouse', 'vilnius', 'wolfsburg', 'zurich']
